In [1]:
import pickle
with open('../Data/allplayerGameLogs.pickle', 'rb') as handle:
  allplayerGameLogs = pickle.load(handle)

In [2]:
import pandas as pd

In [3]:
allplayerGameLogs.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22015,203112,0021500889,"FEB 29, 2016",SAC vs. OKC,L,28,5,9,0.556,...,0,4,2,3,0,0,3,10,-16,1
1,22015,203112,0021500867,"FEB 26, 2016",SAC vs. LAC,L,15,1,1,1.000,...,0,0,0,0,1,0,3,4,-12,1
2,22015,203112,0021500854,"FEB 24, 2016",SAC vs. SAS,L,23,6,9,0.667,...,2,2,0,2,0,1,4,15,13,1
3,22015,203112,0021500842,"FEB 23, 2016",SAC @ DEN,W,13,0,2,0.000,...,3,3,0,0,3,0,1,0,-1,1
4,22015,203112,0021500816,"FEB 19, 2016",SAC vs. DEN,W,15,0,0,0.000,...,3,3,0,0,0,1,1,0,2,1


In [4]:
allplayerGameLogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18964 entries, 0 to 42
Data columns (total 27 columns):
SEASON_ID          18964 non-null object
Player_ID          18964 non-null float64
Game_ID            18964 non-null object
GAME_DATE          18964 non-null object
MATCHUP            18964 non-null object
WL                 18964 non-null object
MIN                18964 non-null float64
FGM                18964 non-null float64
FGA                18964 non-null float64
FG_PCT             18964 non-null float64
FG3M               18964 non-null float64
FG3A               18964 non-null float64
FG3_PCT            18964 non-null float64
FTM                18964 non-null float64
FTA                18964 non-null float64
FT_PCT             18964 non-null float64
OREB               18964 non-null float64
DREB               18964 non-null float64
REB                18964 non-null float64
AST                18964 non-null float64
STL                18964 non-null float64
BLK              

Modify the datetime

In [5]:
allplayerGameLogs['GAME_DATE'] = pd.to_datetime(allplayerGameLogs['GAME_DATE'])

In [6]:
allplayerGameLogs['GAME_DATE'].head()

0   2016-02-29
1   2016-02-26
2   2016-02-24
3   2016-02-23
4   2016-02-19
Name: GAME_DATE, dtype: datetime64[ns]

In [7]:
del allplayerGameLogs['VIDEO_AVAILABLE']

In [8]:
allplayerGameLogs.columns

Index([u'SEASON_ID', u'Player_ID', u'Game_ID', u'GAME_DATE', u'MATCHUP', u'WL',
       u'MIN', u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM',
       u'FTA', u'FT_PCT', u'OREB', u'DREB', u'REB', u'AST', u'STL', u'BLK',
       u'TOV', u'PF', u'PTS', u'PLUS_MINUS'],
      dtype='object')

Add columns to indicate whether the player has double-double or triple-double.

In [9]:
dd = ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['AST']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['REB']>=10))| \
     ((allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10))| \
     ((allplayerGameLogs['REB']>=10) & (allplayerGameLogs['BLK']>=10))| \
     ((allplayerGameLogs['STL']>=10) & (allplayerGameLogs['BLK']>=10))| \
     ((allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['BLK']>=10))

In [10]:
sum(dd)

1517

In [11]:
allplayerGameLogs['DouBL']= dd

In [12]:
ttt = (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['BLK']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['BLK']>=10) & (allplayerGameLogs['REB']>=10)| \
      (allplayerGameLogs['PTS']>=10) & (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['AST']>=10)| \
      (allplayerGameLogs['STL']>=10) & (allplayerGameLogs['AST']>=10) & (allplayerGameLogs['REB']>=10)

In [13]:
sum(ttt)

47

In [14]:
allplayerGameLogs['TriBL']= ttt

calculate the fantasy point for a player to add a column in the data frame

In [15]:
allplayerGameLogs['FanPTs'] = 3.5 * allplayerGameLogs['FG3M'] + 1*allplayerGameLogs['FTM'] \
+ 2*(allplayerGameLogs['FGM']-allplayerGameLogs['FG3M']) \
+ 1.25*allplayerGameLogs['REB']+1.5*allplayerGameLogs['AST'] \
+ 2*allplayerGameLogs['STL']+ 2*allplayerGameLogs['BLK'] \
+ 1.5*allplayerGameLogs['DouBL'] + 3*allplayerGameLogs['TriBL'] \
- 0.5*allplayerGameLogs['TOV']

Test case to verify

In [16]:
davis = allplayerGameLogs[allplayerGameLogs['Player_ID']==203076]

In [17]:
davis[['GAME_DATE','MATCHUP','PTS','AST','REB','BLK','STL','TOV','DouBL','TriBL','FanPTs']]

,GAME_DATE,MATCHUP,PTS,AST,REB,BLK,STL,TOV,DouBL,TriBL,FanPTs
0,2016-02-25,NOP vs. OKC,30,2,6,4,1,2,False,False,50.50
1,2016-02-23,NOP @ WAS,9,3,20,1,2,3,False,False,43.00
2,2016-02-21,NOP @ DET,59,4,20,1,0,2,True,False,93.50
3,2016-02-19,NOP vs. PHI,34,1,7,2,2,2,False,False,51.25
4,2016-02-11,NOP @ OKC,23,2,3,0,2,1,False,False,33.25
5,2016-02-10,NOP vs. UTA,19,5,4,1,0,0,False,False,34.00
6,2016-02-08,NOP @ MIN,27,1,8,0,2,0,False,False,43.00
7,2016-02-06,NOP @ CLE,24,0,11,2,0,2,True,False,42.25
8,2016-02-04,NOP vs. LAL,39,0,11,1,1,1,True,False,57.75
9,2016-02-03,NOP @ SAS,28,4,10,4,4,4,True,False,62.00


Already checked.. it does match the official record from my playing history.

In [94]:
allplayerFantasyGameLogs = allplayerGameLogs.set_index('GAME_DATE')

In [95]:
allplayerFantasyGameLogs = allplayerFantasyGameLogs.sort_index(axis=0)

In [89]:
allplayerFantasyGameLogs.head()

,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,0,...,3,1,0,0,5,0,-21,False,False,6.50
2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,0,...,2,0,2,3,2,5,-1,False,False,16.75
2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,0,...,1,3,0,3,3,8,-8,False,False,20.25
2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,1,...,4,1,0,3,1,3,-14,False,False,10.00
2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,0,...,2,2,1,0,1,17,3,False,False,32.25


Use join to get the corresponding player names

In [53]:
import json
with open('../Data/players.json') as data_file:
    playid_data = json.load(data_file)

In [54]:
playid_df = pd.DataFrame(playid_data)

In [57]:
playid_df['fullName'] = playid_df['firstName'] + ' ' + playid_df['lastName']

In [58]:
playid_df.head()

,firstName,lastName,playerId,teamId,fullName
0,Quincy,Acy,203112,1610612758,Quincy Acy
1,Jordan,Adams,203919,1610612763,Jordan Adams
2,Steven,Adams,203500,1610612760,Steven Adams
3,Arron,Afflalo,201167,1610612752,Arron Afflalo
4,Alexis,Ajinca,201582,1610612740,Alexis Ajinca


In [96]:
allplayerFantasyGameLogs = pd.merge(allplayerFantasyGameLogs.reset_index(), playid_df[['playerId' \
     ,'fullName']], left_on='Player_ID',right_on='playerId', how='left')

In [97]:
del allplayerFantasyGameLogs['playerId']

In [129]:
allplayerFantasyGameLogs.head()

,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,0,...,1,0,0,5,0,-21,False,False,6.50,Nate Robinson
2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,0,...,0,2,3,2,5,-1,False,False,16.75,Timofey Mozgov
2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,0,...,3,0,3,3,8,-8,False,False,20.25,Alonzo Gee
2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,1,...,1,0,3,1,3,-14,False,False,10.00,Steve Blake
2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,0,...,2,1,0,1,17,3,False,False,32.25,Jimmy Butler


In [130]:
with open('../Data/allplayerFantasyGameLogs.pickle', 'wb') as handle:
  pickle.dump(allplayerFantasyGameLogs,handle)

#Helper functions to do EDA and feature extraction

In [100]:
allplayerFantasyGameLogs = allplayerFantasyGameLogs.set_index('GAME_DATE')

In [42]:
# function to generate feature tables, namely training and testing data sets given a game date, where training data set contains all 
# information before the predefined game date

In [43]:
def get_train_test(previous_date ,new_game_date): #format like'2/10/2016'
    train = allplayerFantasyGameLogs.ix[:previous_date]
    test = allplayerFantasyGameLogs.ix[new_game_date]
    return(train, test)

In [101]:
train, test = get_train_test('12/10/2015','12/11/2015')

In [102]:
train.head()

,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,0,...,1,0,0,5,0,-21,False,False,6.50,Nate Robinson
2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,0,...,0,2,3,2,5,-1,False,False,16.75,Timofey Mozgov
2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,0,...,3,0,3,3,8,-8,False,False,20.25,Alonzo Gee
2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,1,...,1,0,3,1,3,-14,False,False,10.00,Steve Blake
2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,0,...,2,1,0,1,17,3,False,False,32.25,Jimmy Butler


In [50]:
#function to aggregate statistics for each player up to a given game date

In [109]:
test = allplayerFantasyGameLogs.ix['10/27/2015':'11/5/2015']

In [110]:
test

,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,...,STL,BLK,TOV,PF,PTS,PLUS_MINUS,DouBL,TriBL,FanPTs,fullName
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2015-10-27,22015,101126,0021500003,NOP @ GSW,L,19,0,1,0.000,0,...,1,0,0,5,0,-21,False,False,6.50,Nate Robinson
2015-10-27,22015,202389,0021500002,CLE @ CHI,L,21,2,6,0.333,0,...,0,2,3,2,5,-1,False,False,16.75,Timofey Mozgov
2015-10-27,22015,202087,0021500003,NOP @ GSW,L,27,4,5,0.800,0,...,3,0,3,3,8,-8,False,False,20.25,Alonzo Gee
2015-10-27,22015,2581,0021500001,DET @ ATL,W,16,1,6,0.167,1,...,1,0,3,1,3,-14,False,False,10.00,Steve Blake
2015-10-27,22015,202710,0021500002,CHI vs. CLE,W,37,6,14,0.429,0,...,2,1,0,1,17,3,False,False,32.25,Jimmy Butler
2015-10-27,22015,203521,0021500002,CLE @ CHI,L,20,3,8,0.375,0,...,0,0,1,1,6,1,False,False,16.50,Matthew Dellavedova
2015-10-27,22015,201567,0021500002,CLE @ CHI,L,35,6,17,0.353,3,...,0,2,2,0,18,1,False,False,38.50,Kevin Love
2015-10-27,22015,201143,0021500001,ATL vs. DET,L,30,6,11,0.545,1,...,2,3,1,1,15,-5,False,False,36.00,Al Horford
2015-10-27,22015,2592,0021500002,CLE @ CHI,L,4,0,2,0.000,0,...,0,1,0,1,0,-7,False,False,2.00,James Jones


In [113]:
test.columns

Index([ u'SEASON_ID',  u'Player_ID',    u'Game_ID',    u'MATCHUP',
               u'WL',        u'MIN',        u'FGM',        u'FGA',
           u'FG_PCT',       u'FG3M',       u'FG3A',    u'FG3_PCT',
              u'FTM',        u'FTA',     u'FT_PCT',       u'OREB',
             u'DREB',        u'REB',        u'AST',        u'STL',
              u'BLK',        u'TOV',         u'PF',        u'PTS',
       u'PLUS_MINUS',      u'DouBL',      u'TriBL',     u'FanPTs',
         u'fullName'],
      dtype='object')

In [116]:
interest_columns = ['fullName','MIN','PTS','FGM','FGA', 'FG3M','FG3A','REB','AST','STL','TOV','PLUS_MINUS','DouBL','TriBL','FanPTs']

In [117]:
test.grouped = test[interest_columns].reset_index().groupby('fullName')

In [127]:
def s(group):
    group['NumGames'] = group['fullName'].count()
    group['AvgFanPTs'] = group['FanPTs'].mean()
    group['AvgPTS'] = group['PTS'].mean()
    group['AvgMIN'] = group['MIN'].mean() 
    group['AvgFGM'] = group['FGM'].mean()
    group['AvgFGA'] = group['FGA'].mean()
    group['AvgFG3M'] = group['FG3A'].mean()
    group['AvgREB'] = group['REB'].mean()
    group['AvgAST'] = group['AST'].mean()
    group['AvgSTL'] = group['STL'].mean()
    group['AvgTOV'] = group['TOV'].mean()
    group['AvgPLUS_MINUS'] = group['PLUS_MINUS'].mean()
    group['NumDouBL'] = group['DouBL'].sum()
    group['NumTriBL'] = group['TriBL'].sum()
       
    return group

So we can get a aggregated table for players' performance up to a specific date. They are useful to serve as our feature variables

In [136]:
Newdf2 = test.grouped.apply(s)
Newdf2 = Newdf2.drop(['GAME_DATE','MIN','PTS','FGM','FGA', 'FG3M','FG3A','REB','AST','STL','TOV','PLUS_MINUS','DouBL','TriBL','FanPTs'],axis=1)
Newdf2

,fullName,NumGames,AvgPTS,AvgMIN,AvgFGM,AvgFGA,AvgFG3M,AvgREB,AvgAST,AvgSTL,AvgTOV,AvgPLUS_MINUS,AvgFanPTs
0,Nate Robinson,2,0.000000,11.500000,0.000000,0.500000,0.500000,0.000000,2.000000,0.500000,0.000000,-10.000000,4.000000
1,Timofey Mozgov,5,7.800000,21.600000,3.000000,6.600000,0.200000,4.800000,1.000000,0.000000,1.600000,-1.200000,17.300000
2,Alonzo Gee,4,3.750000,17.750000,1.750000,3.750000,0.750000,2.500000,0.750000,1.000000,1.250000,-16.250000,9.375000
3,Steve Blake,4,1.250000,14.000000,0.500000,4.250000,2.750000,1.750000,4.250000,0.250000,1.750000,-6.750000,10.062500
4,Jimmy Butler,6,21.666667,36.000000,7.000000,15.000000,3.500000,4.833333,2.666667,2.333333,0.333333,1.333333,39.125000
5,Matthew Dellavedova,5,8.000000,25.200000,3.200000,7.400000,2.800000,3.600000,6.600000,0.600000,1.400000,14.800000,23.400000
6,Kevin Love,5,17.000000,31.600000,6.000000,14.600000,6.200000,11.000000,3.000000,0.000000,1.600000,11.200000,37.550000
7,Al Horford,6,18.000000,32.500000,7.666667,14.166667,3.166667,8.666667,2.500000,0.833333,1.666667,3.000000,38.416667
8,James Jones,5,4.000000,7.200000,1.200000,2.600000,2.000000,0.400000,0.000000,0.000000,0.200000,-3.400000,5.300000
9,Tony Snell,6,6.333333,23.833333,2.166667,5.833333,3.000000,3.333333,1.000000,0.333333,0.666667,3.833333,13.250000


In [151]:
def aggr_stats(date):
    interest_columns = ['fullName','Player_ID','MIN','PTS','FGM','FGA', 'FG3M','FG3A', \
                        'REB','AST','STL','TOV','PLUS_MINUS','DouBL','TriBL','FanPTs']
    tmp = allplayerFantasyGameLogs.ix['10/27/2015':date]
    tmp.grouped = tmp[interest_columns].reset_index().groupby('fullName')
    
    def aggr(group):
        group['NumGames'] = group['fullName'].count()
        group['AvgFanPTs'] = group['FanPTs'].mean()
        group['AvgPTS'] = group['PTS'].mean()
        group['AvgMIN'] = group['MIN'].mean() 
        group['AvgFGM'] = group['FGM'].mean()
        group['AvgFGA'] = group['FGA'].mean()
        group['AvgFG3M'] = group['FG3M'].mean()
        group['AvgFG3A'] = group['FG3A'].mean()
        group['AvgREB'] = group['REB'].mean()
        group['AvgAST'] = group['AST'].mean()
        group['AvgSTL'] = group['STL'].mean()
        group['AvgTOV'] = group['TOV'].mean()
        group['AvgPLUS_MINUS'] = group['PLUS_MINUS'].mean()
        group['NumDouBL'] = group['DouBL'].sum()
        group['NumTriBL'] = group['TriBL'].sum()

        return group
    
    Newdf = tmp.grouped.apply(aggr)
    Newdf = Newdf.drop(['GAME_DATE','MIN','PTS','FGM','FGA', 'FG3M','FG3A','REB','AST','STL','TOV','PLUS_MINUS','DouBL','TriBL','FanPTs'],axis=1)
    
    return(Newdf)

Test

In [152]:
PlayerFeaturesTable_02_29_2016 = aggr_stats('2/29/2016')

In [153]:
PlayerFeaturesTable_02_29_2016.head()

,fullName,Player_ID,NumGames,AvgFanPTs,AvgPTS,AvgMIN,AvgFGM,AvgFGA,AvgFG3M,AvgFG3A,AvgREB,AvgAST,AvgSTL,AvgTOV,AvgPLUS_MINUS,NumDouBL,NumTriBL
0,Nate Robinson,101126,2,4.000000,0.000000,11.500000,0.000000,0.500000,0.000000,0.500000,0.000000,2.000000,0.500000,0.000000,-10.000000,0,0
1,Timofey Mozgov,202389,56,14.223214,6.357143,17.821429,2.732143,4.982143,0.017857,0.125000,4.500000,0.410714,0.285714,0.964286,0.214286,1,0
2,Alonzo Gee,202087,58,11.012931,3.827586,21.448276,1.534483,3.172414,0.258621,0.913793,3.051724,0.948276,0.896552,0.637931,-2.517241,0,0
3,Steve Blake,2581,36,11.430556,4.277778,16.000000,1.527778,3.944444,0.944444,2.611111,1.444444,3.083333,0.416667,1.500000,-1.138889,0,0
4,Jimmy Butler,202710,48,39.682292,22.375000,37.812500,7.437500,16.250000,1.104167,3.333333,5.187500,4.270833,1.687500,1.979167,0.187500,6,0


In [161]:
with open('../Data/PlayerFeaturesTable_02_29_2016.pickle', 'wb') as handle:
  pickle.dump(PlayerFeaturesTable_02_29_2016,handle)

In [49]:
# function to generate a score(PTS) leader board given a game date, the same applies to other 
# attributes, such as 'REB', 'AST', 'FG3', 'BLK', 'PLUS_MINUS'

In [156]:
def get_leader_board(attr, date):
    tmp = allplayerFantasyGameLogs.ix['10/27/2015':date]
    tmp = tmp[['fullName',attr]].reset_index().groupby('fullName').mean()
    result_df = tmp.sort_values([attr], ascending=False)

    return(result_df)

Test 1. leader board for scoring

In [157]:
test1 = get_leader_board('PTS','2/29/2016')

In [158]:
test1.head()

,PTS
fullName,
Stephen Curry,30.678571
James Harden,28.500000
Kevin Durant,27.905660
DeMarcus Cousins,27.306122
Damian Lillard,25.358491


Test 2. leader board for Fantasy Points

In [159]:
get_leader_board('FanPTs','2/29/2016').head(10)

,FanPTs
fullName,
Russell Westbrook,54.112500
Stephen Curry,53.151786
DeMarcus Cousins,51.505102
James Harden,50.995833
Kevin Durant,49.650943
LeBron James,47.350877
Anthony Davis,46.799020
John Wall,46.105932
Damian Lillard,44.198113


In [160]:
get_leader_board('PLUS_MINUS','2/29/2016').head(10)

,PLUS_MINUS
fullName,
Stephen Curry,14.267857
Draymond Green,13.964912
Klay Thompson,12.107143
Kawhi Leonard,9.685185
Jorge Gutierrez,9.000000
Kevin Durant,8.735849
Michael Kidd-Gilchrist,8.285714
Tim Duncan,8.113636
Russell Westbrook,8.066667


In [ ]:
# function to aggregate match-up info given a player by a game date